In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# メルカリ価格予想

## プロセスとして次の順序でデータ分析をやっていきます

## ①業務理解→②データ理解→③前処理→④手法選択→⑤学習→⑥性能評価

# ①業務理解

今回はフリマアプリ最大手のメルカリがもつ大量の売買データを用いて、データ分析を行います。

メルカリでは画像では見分けがつかない同じような見た目の服でも状態によって数倍の値段差が生まれる場合があります。

ほかにも、ただの紙でも有名人がサインを書くだけで、紙は紙でも価値は跳ね上がります。

何が言いたいかというと、何に本当に価値があるのかを知ることは難しいことだということです。

何かを売りたいとき、例えば家の片づけをして出てきた昔の家電を売るとしましょう。

この商品の持つ本当の価値はなかなかわからないと思います。

そこでメルカリが持つたくさんのデータから、売り手に最適な価格提案を提供したいと思っています。

In [ ]:
#モジュールのインポート
# 基本モジュール
import string
import nltk
import re
import numpy as np
import pandas as pd

# 可視化モジュール
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

# 不要な警告を非表示にする
import warnings
warnings.filterwarnings('ignore')

# 標準化モジュール
from scipy.stats import zscore

#その他
from sklearn.feature_extraction import _stop_words
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud


# ②データ理解

In [ ]:
# tsvファイルからPandas DataFrameへ読み込み
train = pd.read_csv('../input/mercari-data/train.tsv',delimiter='\t')
test = pd.read_csv('../input/mercari-data/test_stg2.tsv',delimiter='\t')
train.head(5)

In [ ]:
test.head()

## 今回使うデータの詳細

####   train_id    または   test_id　ーーーーー　　リストのID

#### name　ーーーーー　商品のタイトル。(タイトルに価格に関する情報がある場合（例：$20）はメルカリが事前に削除をして[rm]と置き換えています。)

#### item_condition_id　ーーーーー　商品の状態

#### category_name　ーーーーー　商品のカテゴリ

#### brand_name　ーーーーー　商品のブランドの名前

### price　ーーーーー　売買された価格。これがターゲット変数です。単位は米ドル。

#### shipping　ーーーーー　送料。売り手が支払う場合１、買い手が支払う場合０。

#### item_description　ーーーーー　商品の詳細な説明。タイトルと同様に価格情報がある場合は[rm]と置き換えられています。

今回は説明変数７つと目的変数で構成されています。

In [ ]:
train.info()

In [ ]:
test.info()

今回かなり大きなデータセットになっており、数は下記のようになります

trainデータ　1482535個　　testデータ　　3460725個

ローカルの開発環境ではなかなか厳しそうです

In [ ]:
train.isnull().sum()

brand_name の欠損値が非常に多いです。

次にデータの統計量を確認します。

In [ ]:
train.describe(include='all')

この表の"price"を見ると、売買最高額が2009ドルで売買最低額が0ドルということがわかります。

0ドルの取引も気になりますが深追いすると進まないので次に行きます。

項目ごとに細かく見ていきます。

## price

In [ ]:
plt.hist(train[train['price']<250].price)

ほとんどが50ドル以下の取引になっています

In [ ]:
g=train[train['price']<50]
plt.hist(g.price)

50ドル以下の詳細です。
10ドルから20ドルが一番多いですね。

## name

In [ ]:
print(len(train.name.unique()))
print(len(train.name))

unique値を見ると、同じ名前の出品もあるようです。
名前はあまり重要ではない気もします

## item_condition_id

商品の状態です

１から順に新品～傷有りのように並んでいます

In [ ]:
sns.countplot(data=train, x = 'item_condition_id', label='Count')

新品出品が一番多いです

In [ ]:
dfre=train.groupby("item_condition_id")["price"].mean().reset_index()
dfre

商品の状態は関係あると思ってましたが、あまり関係ないようですね

## category_name

category_nameをより分かりやすくするために下の3段階のカテゴリーに分けます。

category_main	category_sub1	category_sub2

In [ ]:
def transform_category_name(category_name):
    try:
        main, sub1, sub2= category_name.split('/')
        return main, sub1, sub2
    except:
        return np.nan, np.nan, np.nan

train['category_main'], train['category_sub1'], train['category_sub2'] = zip(*train['category_name'].apply(transform_category_name))
test['category_main'], test['category_sub1'], test['category_sub2'] = zip(*test['category_name'].apply(transform_category_name))

In [ ]:
train=train.drop(["category_name"],axis=1)
test=test.drop(["category_name"],axis=1)

In [ ]:
print(len(train.category_main.unique()))
print(len(train.category_sub1.unique()))
print(len(train.category_sub2.unique()))
print(len(test.category_main.unique()))
print(len(test.category_sub1.unique()))
print(len(test.category_sub2.unique()))

In [ ]:
sns.countplot(data=train, y = 'category_main',label='Count')

In [ ]:
N=train['category_main'].value_counts()['Women']
print(N/len(train))

メインのカテゴリーは11個

カテゴリーサブ1は114個

カテゴリーサブ2は865個

取引の約45%はwomenであることがわかりました
やはり主婦さんに人気なんでしょうか

## brand name

ブランドは物の価値を決めるにあたってとても大事です

In [ ]:
df1=train[train["brand_name"].isnull()]

In [ ]:
df2=train.dropna(subset=["brand_name"])

In [ ]:
print("ブランド名なし ："+str(df1["price"].mean()))
print("ブランド名あり ："+str(df2["price"].mean()))

これを見ると、ブランド名があるほうが、やはりつく値段は高いことがわかります。

In [ ]:
print("%d個のブランド名があります" % train['brand_name'].nunique())
train["brand_name"].value_counts().head(10)

商品数が多いブランド上位10個です

日本で人気のブランドもあればあまりなじみのないものもあります。


## shipping

0が送料込み、1が着払いです

In [ ]:
print(train['shipping'].value_counts()[0])
print(train['shipping'].value_counts()[1])

In [ ]:
a=train[train['shipping']==0]
b=train[train['shipping']==1]
print("送料込み ："+str(a["price"].mean()))
print("着払い ："+str(b["price"].mean()))

思った以上に着払いが多かったです
やはり送料込みのほうが送料の分はっきり値段が高くなっています。

## Item Description

この項目は非構造化データであるため、解析は難しいです。
より長い説明が価格を高くするのでしょうか?
より詳細に調べるために文字数を数えてみましょう。

In [ ]:
def wordCount(text):
    # convert to lower case and strip regex
    try:
         # convert to lower case and strip regex
        text = text.lower()
        regex = re.compile('[' +re.escape(string.punctuation) + '0-9\\r\\t\\n]')
        txt = regex.sub(" ", text)
        # tokenize
        # words = nltk.word_tokenize(clean_txt)
        # remove words in stop words
        words = [w for w in txt.split(" ") \
                 if not w in _stop_words.ENGLISH_STOP_WORDS and len(w)>3]
        return len(words)
    except: 
        return 0

In [ ]:
train['desc_len'] = train['item_description'].apply(lambda x: wordCount(x))
test['desc_len'] = test['item_description'].apply(lambda x: wordCount(x))

In [ ]:
train.head()

In [ ]:
df=train.groupby("desc_len")["price"].mean().reset_index()

In [ ]:
plt.plot(df.desc_len,df.price,label="Count")

文字数が多ければ多いほど値段が上がるわけでもなさそうです。

よく出てくる言葉をワードクラウドを使って表示します

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.feature_extraction.text import TfidfVectorizer

text = " ".join(review for review in train.item_description.astype(str))

# Create stopword list:

stopwords = set(STOPWORDS)

wordcloud = WordCloud(stopwords=stopwords, background_color="white", width=700, height=400,min_font_size=15).generate(text)

plt.axis("off")
plt.figure( figsize=(40,20))
plt.tight_layout(pad=0)
plt.imshow(wordcloud, interpolation='bilinear')
plt.show()

よく使われている言葉がわかります。

# ③前処理

### 次に、データタイプが文字列のものを数値に変えていきます。

その前にいらない列をデータフレームから削除したいと思います

In [ ]:
train=train.drop(["item_description"],axis=1)
test=test.drop(["item_description"],axis=1)
train=train.drop(["train_id"],axis=1)


In [ ]:
train.dtypes,test.dtypes

数値に変換する項目は上のobjectとなっているものです

・name

・brand_name

・category_main  

・category_sub1

・category_sub2

In [ ]:
train.brand_name = train.brand_name.astype('category')
train.name = train.name.astype('category')
train.category_main = train.category_main.astype('category')
train.category_sub1 = train.category_sub1.astype('category')
train.category_sub2 = train.category_sub2.astype('category')


test.brand_name = test.brand_name.astype('category')
test.name = test.name.astype('category')
test.category_main = test.category_main.astype('category')
test.category_sub1 = test.category_sub1.astype('category')
test.category_sub2 = test.category_sub2.astype('category')


In [ ]:
train.brand_name = train.brand_name.cat.codes
train.name = train.name.cat.codes
train.category_main = train.category_main.cat.codes
train.category_sub1 = train.category_sub1.cat.codes
train.category_sub2 = train.category_sub2.cat.codes


test.brand_name = test.brand_name.cat.codes
test.name = test.name.cat.codes
test.category_main = test.category_main.cat.codes
test.category_sub1 = test.category_sub1.cat.codes
test.category_sub2 = test.category_sub2.cat.codes


price もlog関数で処理します

In [ ]:
train['price'] = train['price'].apply(lambda x: np.log(x) if x>0 else x)

In [ ]:
train.head()

### これですべて数値に変換できました

#### 各項目の相関を見てみます

In [ ]:
corr_mat =train.corr(method='pearson')

In [ ]:
sns.heatmap(corr_mat,
            vmin=-1.0, #最小値
            vmax=1.0, #最大値
            center=0, #中央値
            annot=True, # True:格子の中に値を表示
            fmt='.2f', #書式設定
             xticklabels=corr_mat.columns.values,#X軸ラベル
            yticklabels=corr_mat.columns.values #Y軸ラベル
           )
plt.show()

少しでも相関が考えられるのはshipping、brand name、category_mainとでてきました

In [ ]:
train

# ③手法選択④学習⑤性能評価

今回は練習もかねて、様々な手法でモデル構築していきたいと思います。
主に中心となるのはアンサンブルモデルです。

## 重回帰分析

In [ ]:
from sklearn import linear_model
reg = linear_model.LinearRegression()

y=train.price
X=train[["shipping","brand_name","category_main"]]
reg.fit(X,y)

In [ ]:
reg.score(X,y)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
X_train= train.drop(['price'], axis=1)
y_train= train.price
X_test=test


## ランダムフォレスト

In [ ]:
# モデルの作成
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
rf = RandomForestRegressor(n_jobs=-1, min_samples_leaf=5, n_estimators=200)
rf.fit(X_train,y_train)
 
# スコアを表示
rf_score=rf.score(X_train,y_train)
print("ランダムフォレストのaccuracyは{0:.4f}".format(rf_score))
print('-'*50)

## 勾配ブースティング

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor
gbcl = GradientBoostingRegressor(n_estimators = 50, learning_rate = 0.05)
gbcl = gbcl.fit(X_train, y_train)
gbcl_score=gbcl.score(X_train , y_train)

print("勾配ブースティングのaccuracyは{0:.4f}".format(gbcl_score))
print('-'*50)

## バギング（Bagging）

In [ ]:
from sklearn.ensemble import BaggingClassifier,BaggingRegressor

bgcl = BaggingRegressor(n_estimators=10, max_samples= .7, bootstrap=True)
bgcl = bgcl.fit(X_train, y_train)
bgcl_score=bgcl.score(X_train, y_train)

print("バギングのaccuracyは{0:.4f}".format(bgcl_score))
print('-'*50)

## アダブースト（Adaboost）

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor

abcl = AdaBoostRegressor( n_estimators= 20)
abcl = abcl.fit(X_train, y_train)


abcl_score=abcl.score(X_train, y_train)
print("アダブーストのaccuracyは{0:.4f}".format(abcl_score))
print('-'*50)


## モデルの比較

In [ ]:
print("ランダムフォレストのaccuracyは{0:.4f}".format(rf_score))
print("アダブーストのaccuracyは{0:.4f}".format(abcl_score))
print("バギングのaccuracyは{0:.4f}".format(bgcl_score))
print("勾配ブースティングのaccuracyは{0:.4f}".format(gbcl_score))

今回バギングのモデルが一番良いスコアでしたが、過学習していると思われるので
ランダムフォレストを一番最適なモデルと考えました。
（実際にkaggle側に提出し結果も見ましたが、バギングよりランダムフォレストのほうがRMSLEは小さかったです。）

よってこの予測モデルで提出ファイルを作りたいと思います。

In [ ]:
test1=test.drop(["test_id"],axis=1)
preds = rf.predict(test1)
 
np.exp(preds)
 
# Numpy配列からpandasシリーズへ変換
preds = pd.Series(np.exp(preds))
 

In [ ]:
# テストデータのIDと予測値を連結
submit = pd.concat([test.test_id, preds], axis=1)
submit

In [ ]:
 # カラム名をメルカリの提出指定の名前をつける
submit.columns = ['test_id', 'price']
 
# 提出ファイルとしてCSVへ書き出し
submit.to_csv('submission.csv', index=False)
 

In [ ]:
pd.read_csv("submission.csv")

正直あまり良い結果が出ませんでした

でもさまざまな方法が試したかったのでスコアの点数はあまり気にしませんでした

次からはしっかりチューニングや前処理をしっかりしてスコアを大事にしていきたいです